In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [3]:
pip install -U regex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 13.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4
    Uninstalling regex-2021.4.4:
      Successfully uninstalled regex-2021.4.4
Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
# from pyspark.ml.feature import MinHashLSH
# from pyspark.ml.feature import CountVectorizer,  IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
# from pyspark import SparkContext
# from pyspark.sql import SQLContext
# from pyspark.sql import Row
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords

In [21]:
pd.set_option("max_colwidth", 100)

In [5]:
from google.cloud import storage
# !pip install gcsfs --upgrade

### Setting Up Helper Functions

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [9]:
def list_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))

In [10]:
# List all files in given COS directory
def list_blobs_pd(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size)), columns=['Name','Size'])

    blobs_df.style.format({"Size": "{:,.0f}"}) 
    
    return blobs_df

In [11]:
# Delete folder from COS bucket
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        blob.delete()

In [18]:
# Reading data from open bucket, avaible to all students
bucket_read = 'msca-bdp-tweets'

# Saving results into individual bucket, students must update to their own bucket
bucket_write = 'msca-bdp-students-bucket/shared_data/kshitijm'

### Accessing Main Database

In [16]:
#!hadoop fs -ls 'gs://msca-bdp-tweets/final_project'

ls: `msca-bdp-students-bucket/shared_data/kshitijm/Final_Project-FiltData1': No such file or directory


In [ ]:
list_blobs_pd(bucket_read,'final_project')

,Name,Size
0,final_project/,0
1,final_project/_SUCCESS,0
2,final_project/part-00000-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json,4500466
3,final_project/part-00001-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json,4107431
4,final_project/part-00002-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json,4672123
...,...,...
50692,final_project/part-50690-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json,11562361
50693,final_project/part-50691-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json,9132693
50694,final_project/part-50692-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json,15376390
50695,final_project/part-50693-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json,8586044


In [3]:
twitter_df= spark.read.json('gs://msca-bdp-tweets/final_project')

23/03/03 04:30:02 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
23/03/03 04:35:27 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
#twitter_df.count()

99994342

In [13]:
twitter_df.limit(5)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries
null,Wed May 25 00:46:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529262624952569856,1529262624952569856,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,nicolesjchung,"{null, Tue May 24...","<a href=""http://t...",RT @nicolesjchung...,1653439596771,false,it's school. they...,"{false, Thu Dec 0...",null
null,Wed May 25 00:46:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529262625057607680,1529262625057607680,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,KingJames,"{null, Tue May 24...","<a href=""http://t...",RT @KingJames: My...,1653439596796,false,My thoughts and p...,"{false, Thu Oct 1...",null
null,Wed May 25 00:46:...,"[27, 122]","{[], null, [], []...",null,null,0,false,low,null,1529262625099419648,1529262625099419648,SaraIsaacson4,1529205912182968323,1529205912182968323,1112921930015956993,1112921930015956993,false,en,"{{[[[-105.053666,...",null,0,null,null,null,null,null,0,0,,Leahgreenb I am m...,null,"<a href=""http://t...",@SaraIsaacson4 @L...,1653439596806,false,@SaraIsaacson4 @L...,"{false, Sun Feb 2...",null
null,Wed May 25 00:46:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529262625296613376,1529262625296613376,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,priore_ashley,"{null, Wed May 25...","<a href=""http://t...",RT @priore_ashley...,1653439596853,false,Gun violence epid...,"{false, Tue Aug 0...",null
null,Wed May 25 00:46:...,null,"{[], null, [], [{...",null,"{[0, 253], {[], n...",0,false,low,null,1529262625166708739,1529262625166708739,null,null,null,null,null,false,en,null,false,0,null,null,null,null,null,0,0,,null,null,"<a href=""http://t...",WATCH LIVE: Presi...,1653439596822,true,WATCH LIVE: Presi...,"{false, Sat Oct 2...",null


In [18]:
#twitter_df.limit(5)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries
null,Mon Nov 07 16:39:...,null,"{[], null, [], []...",null,null,0,false,low,null,1589658862785028097,1589658862785028097,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,LifeNewsHQ,"{null, Mon Nov 07...","<a href=""http://t...",RT @LifeNewsHQ: B...,1667839181978,false,BREAKING: A fede...,"{false, Fri Mar 0...",null
null,Mon Nov 07 16:39:...,null,"{[], null, [], []...",null,null,0,false,low,null,1589658863376429056,1589658863376429056,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,robbystarbuck,"{null, Sun Nov 06...","<a href=""http://t...",RT @robbystarbuck...,1667839182119,false,This is Dana Ness...,"{false, Tue Apr 2...",null
null,Mon Nov 07 16:39:...,null,"{[], null, [], []...",null,null,0,false,low,null,1589658863489675264,1589658863489675264,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,MarinaPurkiss,"{null, Mon Nov 07...","<a href=""https://...",RT @MarinaPurkiss...,1667839182146,false,Many of those in ...,"{false, Tue Feb 0...",null
null,Mon Nov 07 16:39:...,null,"{[], null, [], []...",null,null,0,false,low,null,1589658864173355008,1589658864173355008,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,twit_fluffy,"{null, Mon Nov 07...","<a href=""https://...",RT @twit_fluffy: ...,1667839182309,false,A new art that I ...,"{false, Sun Nov 0...",null
null,Mon Nov 07 16:39:...,null,"{[], null, [], []...",null,null,0,false,low,null,1589658865355829248,1589658865355829248,null,null,null,null,null,true,en,null,null,0,"{null, Sun Nov 06...",1589202057168359425,1589202057168359425,{twitter.com/BBCN...,Keir Starmer: Too...,0,0,RT,NesrineMalik,"{null, Sun Nov 06...","<a href=""https://...",RT @NesrineMalik:...,1667839182591,false,There is actually...,"{false, Fri Nov 0...",null


In [14]:
#twitter_df.printSchema()

### First Filtering

In [9]:
twitter_df_filt = twitter_df\
.withColumn('tweet_text', lower('tweet_text'))\
.withColumn('stripped', regexp_replace(col("tweet_text"),"[\$#,&%\".]",""))

In [ ]:
twitter_df_filt = twitter_df_filt.filter(twitter_df_filt.lang == 'en')
twitter_df_filt.count()

In [10]:
edu_words = ["primary school","schools","education","k12","high school","teacher","higher secondary","higher education",
             "senior secondary","sophomore","math","mathematics","science","physics","chemistry","biology","humanities",
             "history","philosophy","alma mater","academia","educating","teaching","curriculum","online learning",
             "educational","textbook","kindergarten","schooling","k-12","social-emotional learning","training","knowledge",
             "scholarship","literacy","schooling","tuition","undergraduate","academic","course","curriculum","graduate","stem","phd",
             "classwork","classroom","preschool","educationist","syllabus","middle school","secondary school","undergrad","book ban",
             "student loan","student aid","school shootings","school shooting"]
'|'.join(edu_words)


'primary school|schools|education|k12|high school|teacher|higher secondary|higher education|senior secondary|sophomore|math|mathematics|science|physics|chemistry|biology|humanities|history|philosophy|alma mater|academia|educating|teaching|curriculum|online learning|educational|textbook|kindergarten|schooling|k-12|social-emotional learning|training|knowledge|scholarship|literacy|schooling|tuition|undergraduate|academic|course|curriculum|graduate|stem|phd|classwork|classroom|preschool|educationist|syllabus|middle school|secondary school|undergrad|book ban|student loan|student aid|school shootings|school shooting'

In [11]:
twitter_df_filt = twitter_df_filt.filter(col('tweet_text').rlike('|'.join(edu_words)))

In [ ]:
twitter_df_filt.count()

23/03/03 05:38:43 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1677802855224_0004_01_000013 on host: hub-msca-bdp-dphub-students-kshitijm-sw-6kvx.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2023-03-03 05:38:43.937]Container killed on request. Exit code is 143
[2023-03-03 05:38:43.941]Container exited with a non-zero exit code 143. 
[2023-03-03 05:38:43.942]Killed by external signal
.
23/03/03 05:38:43 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 13 for reason Container from a bad node: container_1677802855224_0004_01_000013 on host: hub-msca-bdp-dphub-students-kshitijm-sw-6kvx.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2023-03-03 05:38:43.937]Container killed on request. Exit code is 143
[2023-03-03 05:38:43.941]Container exited with a non-zero exit code 143. 
[2023-03-03 05:38:43.942]Killed by external signal
.
23/03/03 05:38:43 ERRO

39848346

In [13]:
twitter_df_filt.write.format("parquet").\
mode('overwrite').\
save('gs://' + 'msca-bdp-students-bucket/shared_data/kshitijm' + '/Final_Project-FiltData0')

In [ ]:
education_text_sample = twitter_df_filt.sample(withReplacement=False, fraction=0.025, seed=12345)
education_text_sample.limit(40)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries,stripped
null,Sat Oct 22 08:14:...,null,"{[], null, [], []...",null,null,0,false,low,null,1583733489350111232,1583733489350111232,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,sirmobbsalot,"{null, Thu Oct 20...","<a href=""http://t...",RT @sirmobbsalot:...,1666426462863,false,process the teach...,"{false, Sun Nov 0...",null,process the teach...
null,Sat Oct 22 08:14:...,null,"{[], null, [], []...",null,null,0,false,low,null,1583733538888638464,1583733538888638464,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,DrFrankRoberts,"{null, Fri Oct 21...","<a href=""http://t...",RT @DrFrankRobert...,1666426474674,false,my rule: any stud...,"{false, Thu Sep 0...",null,my rule: any stud...
null,Sat Oct 22 08:15:...,null,"{[], null, [], []...",null,null,0,false,low,null,1583733735790235648,1583733735790235648,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,IAGovernor,"{null, Thu Oct 20...","<a href=""http://t...",RT @IAGovernor: T...,1666426521619,false,the covid vaccine...,"{false, Mon Jul 1...",null,the covid vaccine...
null,Sat Oct 22 08:16:...,null,"{[{[28, 43], ONEP...",null,null,0,false,low,null,1583733931161317376,1583733931161317376,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,AmericaNewsroom,"{null, Wed Oct 19...","<a href=""http://t...",RT @AmericaNewsro...,1666426568199,false,watch: #onepillca...,"{false, Sat Oct 2...",null,watch: onepillcan...
null,Sat Oct 22 08:16:...,null,"{[], null, [], []...",null,null,0,false,low,null,1583734006583300097,1583734006583300097,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,MailOnline,"{null, Sat Oct 22...","<a href=""http://t...",RT @MailOnline: H...,1666426586181,false,horrific moment f...,"{false, Thu Feb 1...",null,horrific moment f...
null,Sat Oct 22 08:16:...,"[14, 140]","{[], null, [], [{...",null,"{[14, 293], {[{[2...",0,false,low,null,1583734094390665216,1583734094390665216,DmytroKuleba,1583726855819235329,1583726855819235329,2885493519,2885493519,false,en,null,null,0,null,null,null,null,null,0,0,,null,null,"<a href=""http://t...",@DmytroKuleba Und...,1666426607116,true,@dmytrokuleba und...,"{false, Fri Oct 0...",null,@dmytrokuleba und...
null,Sat Oct 22 08:17:...,null,"{[], null, [], []...",null,null,0,false,low,null,1583734263614099457,1583734263614099457,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,adjiboussodieng,"{null, Fri Oct 21...","<a href=""http://t...",RT @adjiboussodie...,1666426647462,false,it's grad school ...,"{false, Wed Dec 0...",null,it's grad school ...
null,Sat Oct 22 08:20:...,null,"{[], null, [], []...",null,null,0,false,low,null,1583735117549572096,1583735117549572096,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,_oluwaseun9,"{null, Sat Oct 22...","<a href=""http://t...",RT @_oluwaseun9: ...,1666426851056,false,another big oppor...,"{false, Thu Jul 0...",null,another big oppor...
null,Sat Oct 22 08:21:...,null,"{[], null, [], [{...",null,"{[0, 156], {[], n...",0,false,low,null,1583735183085166593,1583735183085166593,null,null,null,null,null,false,en,null,false,0,null,null,null,null,null,0,0,,null,null,"<a href=""http://t...",thinking about wh...,1666426866681,true,thinking about wh...,"{false, Sun Aug 0...",null,thinking about wh...
null,Sat Oct 22 08:21:...,null,"{[], null, [], [{...",null,null,0,false,low,null,15

In [28]:
education_text_sample.count()

996858

In [27]:
education_text_sample.select('tweet_text').show(40,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tweet_text                                                                                                                                                                                                                                                                                                      |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|fun fact: in germany, homeschooling is effectively prohibited. you must send y

## Filtering 2 - Removing sports words

In [1]:
twitter_df_filt= spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/kshitijm/Final_Project-FiltData0')

23/03/03 13:14:08 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [2]:
dict_rm = ["baseball","volleyball","association football","varsity","high school softball","playoff","varsity sports",
           "university sports","vs.","college football","college basketball","basketball","track and field","swimming",
           "lacrosse","cricket","handball","professional sports","ultimate frisbee","frisbee","sport","sports","sporting",
           "competition","competitions","tournament","tournaments","championship","championships","march madness",
           "ncaa","national collegiate athletic association","games","game","matches","cheerleaders","cheerleader","mascot",
          "team color","athletic conference","athletic conferences","athletics department"]

#score, scores

dict_rm1='|'.join(["(" + c +")" for c in dict_rm])

In [3]:
twitter_df_filt2 = twitter_df_filt.where(~twitter_df_filt['tweet_text'].rlike(dict_rm1))

In [7]:
twitter_df_filt2.count()

36344266

In [ ]:
twitter_df_filt2.write.format("parquet").\
mode('overwrite').\
save('gs://' + 'msca-bdp-students-bucket/shared_data/kshitijm' + '/Final_Project-FiltData1')

### Saving a sample for further analysis

In [5]:
twitter_df_filt2_sample = twitter_df_filt2.sample(withReplacement=False, fraction=0.025, seed=12345)
twitter_df_filt2_sample.write.format("parquet").\
mode('overwrite').\
save('gs://' + 'msca-bdp-students-bucket/shared_data/kshitijm' + '/Final_Project-FiltData1_Sample')

23/03/03 13:48:35 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1677847511085_0002_01_000025 on host: hub-msca-bdp-dphub-students-kshitijm-sw-qspf.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2023-03-03 13:48:35.923]Container killed on request. Exit code is 143
[2023-03-03 13:48:35.924]Container exited with a non-zero exit code 143. 
[2023-03-03 13:48:35.926]Killed by external signal
.
23/03/03 13:48:35 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1677847511085_0002_01_000026 on host: hub-msca-bdp-dphub-students-kshitijm-sw-qspf.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2023-03-03 13:48:35.924]Container killed on request. Exit code is 143
[2023-03-03 13:48:35.926]Container exited with a non-zero exit code 143. 
[2023-03-03 13:48:35.927]Killed by external signal
.
23/03/03 13:48:35 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Reques